# Classification Exercise

We'll be working with some California Census Data, we'll be trying to use various features of an individual to predict what class of income they belogn in (>50k or <=50k). 

Here is some information about the data:

<table>
<thead>
<tr>
<th>Column Name</th>
<th>Type</th>
<th>Description</th>
</tr>
</thead>
<tbody>
<tr>
<td>age</td>
<td>Continuous</td>
<td>The age of the individual</td>
</tr>
<tr>
<td>workclass</td>
<td>Categorical</td>
<td>The type of employer the  individual has (government,  military, private, etc.).</td>
</tr>
<tr>
<td>fnlwgt</td>
<td>Continuous</td>
<td>The number of people the census  takers believe that observation  represents (sample weight). This  variable will not be used.</td>
</tr>
<tr>
<td>education</td>
<td>Categorical</td>
<td>The highest level of education  achieved for that individual.</td>
</tr>
<tr>
<td>education_num</td>
<td>Continuous</td>
<td>The highest level of education in  numerical form.</td>
</tr>
<tr>
<td>marital_status</td>
<td>Categorical</td>
<td>Marital status of the individual.</td>
</tr>
<tr>
<td>occupation</td>
<td>Categorical</td>
<td>The occupation of the individual.</td>
</tr>
<tr>
<td>relationship</td>
<td>Categorical</td>
<td>Wife, Own-child, Husband,  Not-in-family, Other-relative,  Unmarried.</td>
</tr>
<tr>
<td>race</td>
<td>Categorical</td>
<td>White, Asian-Pac-Islander,  Amer-Indian-Eskimo, Other, Black.</td>
</tr>
<tr>
<td>gender</td>
<td>Categorical</td>
<td>Female, Male.</td>
</tr>
<tr>
<td>capital_gain</td>
<td>Continuous</td>
<td>Capital gains recorded.</td>
</tr>
<tr>
<td>capital_loss</td>
<td>Continuous</td>
<td>Capital Losses recorded.</td>
</tr>
<tr>
<td>hours_per_week</td>
<td>Continuous</td>
<td>Hours worked per week.</td>
</tr>
<tr>
<td>native_country</td>
<td>Categorical</td>
<td>Country of origin of the  individual.</td>
</tr>
<tr>
<td>income</td>
<td>Categorical</td>
<td>"&gt;50K" or "&lt;=50K", meaning  whether the person makes more  than \$50,000 annually.</td>
</tr>
</tbody>
</table>

### THE DATA

** Read in the census_data.csv data with pandas**

In [16]:
import pandas as pd

census = pd.read_csv('census_data.csv')
census.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


** TensorFlow won't be able to understand strings as labels, you'll need to use pandas .apply() method to apply a custom function that converts them to 0s and 1s. This might be hard if you aren't very familiar with pandas, so feel free to take a peek at the solutions for this part.**

** Convert the Label column to 0s and 1s instead of strings.**

In [17]:
def label_01(label):
    if label == ' <=50K':
        return 0
    else:
        return 1

In [18]:
census['income_bracket'] = census['income_bracket'].apply(label_01)
census.head()

,age,workclass,education,education_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_bracket
0,39,State-gov,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


### Perform a Train Test Split on the Data

In [19]:
from sklearn.model_selection import train_test_split

xdata = census.drop('income_bracket',axis = 1)
ydata = census['income_bracket']

xtrain, xtest, ytrain, ytest = train_test_split(xdata, ydata, test_size = 0.3, random_state=101)

### Create the Feature Columns for tf.esitmator

** Take note of categorical vs continuous values! **

** Import Tensorflow **

In [20]:
import tensorflow as tf

** Create the tf.feature_columns for the categorical values. Use vocabulary lists or just use hash buckets. **

In [21]:
gender = tf.feature_column.categorical_column_with_vocabulary_list("gender", ['Female','Male'])
occupation = tf.feature_column.categorical_column_with_hash_bucket("occupation", hash_bucket_size = 1000)
marital_status = tf.feature_column.categorical_column_with_hash_bucket("marital_status", hash_bucket_size = 1000)
relationship = tf.feature_column.categorical_column_with_hash_bucket("relationship", hash_bucket_size = 1000)
education = tf.feature_column.categorical_column_with_hash_bucket("education", hash_bucket_size = 1000)
workclass = tf.feature_column.categorical_column_with_hash_bucket("workclass", hash_bucket_size = 1000)
native_country = tf.feature_column.categorical_column_with_hash_bucket("native_country", hash_bucket_size = 1000)

** Create the continuous feature_columns for the continuous values using numeric_column **

In [22]:
age  = tf.feature_column.numeric_column("age")
education_num  = tf.feature_column.numeric_column("education_num")
capital_gain  = tf.feature_column.numeric_column("capital_gain")
capital_loss  = tf.feature_column.numeric_column("capital_loss")
hours_per_week  = tf.feature_column.numeric_column("hours_per_week")

** Put all these variables into a single list with the variable name feat_cols **

In [23]:
feat_cols = [gender,occupation,marital_status,relationship,education,workclass,native_country,
             age,education_num,capital_gain,capital_loss,hours_per_week]

### Create Input Function

** Batch_size is up to you. But do make sure to shuffle!**

In [24]:
infunc = tf.estimator.inputs.pandas_input_fn(x = xtrain, y = ytrain, batch_size = 100, num_epochs = None, shuffle = True)

#### Create your model with tf.estimator

**Create a LinearClassifier.(If you want to use a DNNClassifier, keep in mind you'll need to create embedded columns out of the cateogrical feature that use strings, check out the previous lecture on this for more info.)**

In [25]:
model = tf.estimator.LinearClassifier(feature_columns = feat_cols)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\1\\AppData\\Local\\Temp\\tmpn31xju63', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000206FE64BC50>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


** Train your model on the data, for at least 5000 steps. **

In [26]:
model.train(input_fn = infunc, steps = 5000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\1\AppData\Local\Temp\tmpn31xju63\model.ckpt.
INFO:tensorflow:loss = 69.31472, step = 1
INFO:tensorflow:global_step/sec: 111.051
INFO:tensorflow:loss = 35.329025, step = 101 (0.902 sec)
INFO:tensorflow:global_step/sec: 191.681
INFO:tensorflow:loss = 630.91516, step = 201 (0.523 sec)
INFO:tensorflow:global_step/sec: 187.725
INFO:tensorflow:loss = 2628.313, step = 301 (0.537 sec)
INFO:tensorflow:global_step/sec: 186.326
INFO:tensorflow:loss = 402.00027, step = 401 (0.533 sec)
INFO:tensorflow:global_step/sec: 183.591
INFO:tensorflow:loss = 782.5785, step = 501 (0.545 sec)
INFO:tensorflow:global_step/sec: 125.228
INFO:tensorflow:loss = 95.28273, step = 601 (0.801 sec)
INFO:tensorflow:global_step/sec: 152

### Evaluation

** Create a prediction input function. Remember to only supprt X_test data and keep shuffle=False. **

In [27]:
predsfunc = tf.estimator.inputs.pandas_input_fn(x = xtest, batch_size = len(xtest), shuffle = False)

** Use model.predict() and pass in your input function. This will produce a generator of predictions, which you can then transform into a list, with list() **

In [28]:
preds = model.predict(input_fn = predsfunc)
predictions = list(preds)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\1\AppData\Local\Temp\tmpn31xju63\model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.


In [29]:
predsfinal = [pred['class_ids'][0] for pred in predictions]

** Import classification_report from sklearn.metrics and then see if you can figure out how to use it to easily get a full report of your model's performance on the test data. **

In [30]:
from sklearn.metrics import classification_report
print(classification_report(ytest,predsfinal))
classification_report

             precision    recall  f1-score   support

          0       0.88      0.90      0.89      7436
          1       0.65      0.60      0.62      2333

avg / total       0.82      0.83      0.83      9769



<function sklearn.metrics.classification.classification_report(y_true, y_pred, labels=None, target_names=None, sample_weight=None, digits=2)>

# Great Job!